In [1]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---
import pandas as pd
import numpy as np
import joblib 
import os 

# Bibliotecas de Pré-processamento e Métricas
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Modelo de Árvore
from sklearn.tree import DecisionTreeClassifier 

print("Bibliotecas importadas e prontas para Modelos de Árvore (Pima Indians Diabetes).")

Bibliotecas importadas e prontas para Modelos de Árvore (Pima Indians Diabetes).


In [2]:
# --- CÉLULA 2: Carregamento de Dados Pima (Arquivos Salvos no Notebook 03) ---

DATA_DIR = 'data'

# Carregamento dos conjuntos padronizados e separados 
try:
    # Para modelos de árvore, podemos usar os dados não-scaled (X_train) ou scaled (X_train_scaled)
    # Usaremos os scaled para consistência, embora a árvore de decisão não seja sensível à escala.
    X_train_scaled = joblib.load(os.path.join(DATA_DIR, 'X_train_pima_scaled.joblib'))
    X_test_scaled = joblib.load(os.path.join(DATA_DIR, 'X_test_pima_scaled.joblib'))
    y_train = joblib.load(os.path.join(DATA_DIR, 'y_train_pima.joblib'))
    y_test = joblib.load(os.path.join(DATA_DIR, 'y_test_pima.joblib'))
    print("Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.")
except Exception as e:
    print(f"ERRO: Não foi possível carregar os arquivos. Execute a Célula 2 do Notebook 03 primeiro. Erro: {e}")

print(f"Shape do X_train: {X_train_scaled.shape}")

Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.
Shape do X_train: (537, 8)


In [3]:
# --- CÉLULA 3: Decision Tree Classifier ---

# 1. Definir o range de hiperparâmetros
param_grid_dt = {
    'max_depth': [3, 5, 10, 15],  # Profundidade máxima
    'min_samples_split': [2, 5, 10] # Mínimo de amostras para dividir
}

# 2. Configurar o GridSearchCV
dt = DecisionTreeClassifier(random_state=42)
dt_grid = GridSearchCV(
    estimator=dt,
    param_grid=param_grid_dt,
    scoring='roc_auc', # Otimizamos para AUC-ROC
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
dt_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_dt_model = dt_grid.best_estimator_
MODEL_DIR = 'models'
MODEL_PATH_DT = os.path.join(MODEL_DIR, 'decision_tree_classifier_pima.joblib')
joblib.dump(best_dt_model, MODEL_PATH_DT)
print(f"\nModelo Decision Tree Classifier otimizado e salvo em: {MODEL_PATH_DT}")

# 5. Avaliação
y_pred_proba_dt = best_dt_model.predict_proba(X_test_scaled)[:, 1] # Probabilidade da Classe 1
y_pred_dt = best_dt_model.predict(X_test_scaled) # Previsão da classe

auc_dt = roc_auc_score(y_test, y_pred_proba_dt)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"\nDecision Tree: Melhor Parâmetro encontrado: {dt_grid.best_params_}")
print(f"Decision Tree: Acurácia (Teste): {accuracy_dt:.4f}")
print(f"Decision Tree: AUC-ROC (Teste): {auc_dt:.4f}")
print("\nRelatório de Classificação (Decision Tree):")
print(classification_report(y_test, y_pred_dt))


Modelo Decision Tree Classifier otimizado e salvo em: models\decision_tree_classifier_pima.joblib

Decision Tree: Melhor Parâmetro encontrado: {'max_depth': 5, 'min_samples_split': 10}
Decision Tree: Acurácia (Teste): 0.7619
Decision Tree: AUC-ROC (Teste): 0.8149

Relatório de Classificação (Decision Tree):
              precision    recall  f1-score   support

           0       0.76      0.93      0.84       150
           1       0.78      0.44      0.57        81

    accuracy                           0.76       231
   macro avg       0.77      0.69      0.70       231
weighted avg       0.77      0.76      0.74       231

